In [1]:
using Revise

In [165]:
using RigidBodyDynamics
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyTreeInspector
using Gurobi
import StochasticOptimization
using Plots
using ProgressMeter

In [7]:
import LCPSim
import LearningMPC
import BoxValkyries
import Nets

In [ ]:
# reload("LCPSim")
reload("LearningMPC")
reload("BoxValkyries")

In [176]:
boxval = BoxValkyries.BoxValkyrie()
mechanism = boxval.mechanism
xstar = BoxValkyries.nominal_state(boxval)

basevis = Visualizer()[:box_robot]
setgeometry!(basevis, boxval)
settransform!(basevis[:robot], xstar)

mpc_params = LearningMPC.MPCParams(
    Δt=0.04,
    horizon=1,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, TimeLimit=120, MIPGap=1e-1, MIPGapAbs=5, FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

feet = findbody.(mechanism, ["rf", "lf"])
contacts = [Point3D(default_frame(body), 0., 0, 0) for body in feet]
Q, R = BoxValkyries.default_costs(xstar)
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, contacts, mpc_params.Δt)
LearningMPC.zero_element!(lqrsol, 1)

hidden_widths = [16, 16, 16, 16]
activation = Nets.leaky_relu
net = LearningMPC.control_net(mechanism, hidden_widths, activation)

net_controller = x -> Nets.predict(net, state_vector(x))

mpc_controller = LearningMPC.MPCController(mechanism, 
    boxval.environment, mpc_params, lqrsol, 
    [net_controller, lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}()
playback_sink = LearningMPC.PlaybackSink(basevis[:robot], mpc_params.Δt)

mpc_controller.callback = LearningMPC.call_each(
    sample_sink,
    playback_sink)

live_viewer = LearningMPC.live_viewer(mechanism, basevis[:robot])

dagger_controller = LearningMPC.call_each(
    LearningMPC.dagger_controller(
        mpc_controller,
        net_controller,
        0.2),
    live_viewer
    )

termination = x -> begin
    (configuration(x)[1] < -1 ||
     configuration(x)[2] < 0.5 || 
     configuration(x)[3] > 1.2 || 
     configuration(x)[3] < -1.2)
end

dataset = LearningMPC.Dataset(lqrsol)

updater = Nets.adam_updater(net)

loss = LearningMPC.sensitive_loss(net, 0.1)
adam_opts = Nets.AdamOpts(learning_rate=0.001, batch_size=1)

function collect!(data::Vector{<:LearningMPC.Sample})
    empty!(sample_sink)
    LearningMPC.randomize!(x0, xstar, 0.1, 0.5)
    results = LCPSim.simulate(x0, 
        dagger_controller,
        boxval.environment, mpc_params.Δt, 100, 
        mpc_params.lcp_solver;
        termination=termination);
    append!(data, sample_sink.samples)
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only


collect! (generic function with 1 method)

In [177]:
x0 = MechanismState{Float64}(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [178]:
all_losses(net, dataset) = (LearningMPC.training_loss(net, dataset),
                            LearningMPC.validation_loss(net, dataset))

all_losses (generic function with 1 method)

In [ ]:
losses = Tuple{Float64, Float64}[]

@showprogress for i in 1:100
    for i in 1:2
        collect!(dataset.training_data)
    end
    collect!(dataset.testing_data)
    collect!(dataset.validation_data);
    
    for i in 1:5
        Nets.adam_update!(net.params.data, updater, loss, 
            LearningMPC.features.(dataset.training_data), adam_opts);
        push!(losses, all_losses(net, dataset))
    end
    
end

Progress:   6%|██                                       |  ETA: 0:32:36

captured: ErrorException("Unrecognized solution status: loaded")
captured: ErrorException("Unrecognized solution status: loaded")


Progress:  10%|████                                     |  ETA: 0:32:59

captured: ErrorException("Unrecognized solution status: loaded")
captured: ErrorException("Unrecognized solution status: loaded")


Progress:  12%|█████                                    |  ETA: 0:32:53

...

captured: ErrorException("Unrecognized solution status: loaded")


Progress:  62%|█████████████████████████                |  ETA: 0:20:46

captured: ErrorException("Unrecognized solution status: loaded")


Progress:  64%|██████████████████████████               |  ETA: 0:19:58

captured: ErrorException("Unrecognized solution status: loaded")


In [ ]:
plt = plot(first.(losses))
plot!(plt, last.(losses))
plt

In [83]:

LearningMPC.randomize!(x0, xstar, 0.1, 0.5)
results = LCPSim.simulate(x0, 
    LearningMPC.call_each(dagger_controller, live_viewer),
    boxval.environment, mpc_params.Δt, 200, mpc_params.lcp_solver);

captured: ErrorException("Unrecognized solution status: loaded")


In [84]:
length(sample_sink.samples)

37